In [2]:
import anndata
import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.model_selection import train_test_split

from pmvae.model import PMVAE
from pmvae.train import train

In [3]:
def remove_unneeded_keys_kang(data):
    data.obs = data.obs[['condition', 'cell_type']]
    data.uns = dict()
    data.obsm = None
    data.varm = None
    
    return

def load_annotations(gmt, genes, min_genes=13):
    genesets = load_gmt_genesets(gmt, genes, min_genes)
    annotations = pd.DataFrame(False, index=genes, columns=genesets.keys())
    for key, genes in genesets.items():
        annotations.loc[genes, key] = True

    return annotations

def load_gmt_genesets(path, symbols=None, min_genes=10):
    lut = dict()
    for line in open(path, 'r'):
        key, _, *genes = line.strip().split()
        if symbols is not None:
            genes = symbols.intersection(genes).tolist()
        if len(genes) < min_genes:
            continue
        lut[key] = genes
        
    return lut


In [4]:
data = anndata.read('../data/kang_count.h5ad')
remove_unneeded_keys_kang(data)
data.varm['annotations'] = load_annotations(
    '../data/c2.cp.reactome.v4.0.symbols.gmt',
    data.var_names)


In [4]:
membership_mask = data.varm['annotations'].values.astype(bool).T
trainset, testset = train_test_split(
    data.X,
    test_size=0.25,
    shuffle=True,
    random_state=0
)

batch_size = 256
trainset = tf.data.Dataset.from_tensor_slices(trainset)
trainset = trainset.shuffle(5 * batch_size).batch(batch_size)


In [5]:
model = PMVAE(
    membership_mask, 4, [12],
    add_auxiliary_module=True,
    beta=1e-05,
    kernel_initializer='he_uniform',
    bias_initializer='zero',
    activation='elu',
)

opt = tf.keras.optimizers.Adam(learning_rate=0.001)


In [ ]:
# This should take ~1hr on a GPU (first iter takes much longer than rest)
history = train(model, opt, trainset, testset, nepochs=1200)

 41%|████▏     | 495/1200 [27:21<38:17,  3.26s/it] 

In [11]:
outputs = model.call(data.X)

In [33]:
from sklearn.manifold import TSNE

def extract_pathway_cols(df, pathway):
    mask = df.columns.str.startswith(pathway + '-')
    
    return df.loc[:, mask]

def embeddings_to_df(codes, terms, index, add_auxiliary=True):
    terms = list(terms)
    if add_auxiliary:
        terms.append('AUXILIARY')
    terms = pd.Series(terms)
    
    latent_dim_per_pathway = codes.shape[1] // terms.size
    term_index = np.tile(range(latent_dim_per_pathway), terms.size).astype(str)
    terms = terms.repeat(latent_dim_per_pathway) + '-' + term_index.astype(str)
    
    return pd.DataFrame(codes, columns=terms.values, index=index)

outdir = Path('../results')
outdir.mkdir(exist_ok=True, parents=True)

recons = anndata.AnnData(
    pd.DataFrame(
        outputs.global_recon.numpy(),
        index=data.obs_names,
        columns=data.var_names),
    obs=data.obs,
    varm=data.varm,
)

recons.obsm['codes'] = embeddings_to_df(
    outputs.z.numpy(),
    data.varm['annotations'].columns,
    data.obs_names)

recons.write(outdir/'recons.h5ad')

In [ ]:
pathways = [
    'REACTOME_INTERFERON_ALPHA_BETA_SIGNALING',
    'REACTOME_CYTOKINE_SIGNALING_IN_IMMUNE_SYSTEM',
    'REACTOME_TCR_SIGNAL',
    'REACTOME_CELL_CYCLE']

for key in pathways:
    codes = extract_pathway_cols(recons.obsm['codes'], key)
    recons.obsm[f'code_tsne-{key}'] = TSNE().fit_transform(codes.values)

recons.write(outdir/'recons.h5ad')    